# Gourmet's dilemma

In [1]:
! pip install gurobipy

^C


In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np

Import and format data

In [35]:
df = pd.read_csv('OPT - parameter values - csv.csv')
df

df['f'] = 2*df['f']

# Transform each column into a dictionary where the key is the index and the value the value in the column
param_dicts = {col: df[col].to_dict() for col in df.columns}
print(param_dicts.keys())
param_dicts

dict_keys(['food_names', 'r', 'u', 'w', 'p', 'f'])


{'food_names': {0: 'Oatmeal',
  1: 'Boiled egg',
  2: 'Strawberry',
  3: 'Apple',
  4: 'Scrambled egg',
  5: 'Bacon',
  6: 'Sausage',
  7: 'Pancake with syrup',
  8: 'French toast',
  9: 'Croissant',
  10: 'Plain yogurt',
  11: 'Strawberry jam',
  12: 'Slice of cheese',
  13: 'Smoked salmon',
  14: 'Donut'},
 'r': {0: 3,
  1: 2,
  2: 5,
  3: 2,
  4: 3,
  5: 5,
  6: 4,
  7: 6,
  8: 4,
  9: 7,
  10: 2,
  11: 3,
  12: 3,
  13: 8,
  14: 4},
 'u': {0: 6,
  1: 7,
  2: 9,
  3: 6,
  4: 7,
  5: 9,
  6: 8,
  7: 9,
  8: 8,
  9: 8,
  10: 5,
  11: 7,
  12: 6,
  13: 8,
  14: 9},
 'w': {0: 0.25,
  1: 0.05,
  2: 0.1,
  3: 0.15,
  4: 0.15,
  5: 0.05,
  6: 0.08,
  7: 0.2,
  8: 0.15,
  9: 0.1,
  10: 0.15,
  11: 0.02,
  12: 0.03,
  13: 0.05,
  14: 0.2},
 'p': {0: 0.1,
  1: 0.1,
  2: 0.05,
  3: 0.05,
  4: 0.2,
  5: 0.5,
  6: 0.4,
  7: 0.3,
  8: 0.3,
  9: 0.6,
  10: 0.1,
  11: 0.2,
  12: 0.4,
  13: 0.2,
  14: 0.8},
 'f': {0: 0.8,
  1: 0.6,
  2: 1.2,
  3: 0.8,
  4: 1.4,
  5: 1.6,
  6: 1.8,
  7: 2.4,
  8: 2.4

Optimization model

In [40]:
function_types = ['logarithmic', 'linear']

df_results = pd.DataFrame({'food_names':df['food_names']}) # results

for alpha, function_type in [(0.5,'logarithmic'), (1,'logarithmic'), (1,'linear'), (2.0,'logarithmic')]:
    print(alpha, function_type)
    # Initialize m
    m = ConcreteModel()

    # Index set
    m.I = RangeSet(0, len(param_dicts['food_names'])-1) # number of foods

    # Variables
    m.x = Var(m.I, within=NonNegativeIntegers, initialize=0)

    # Parameters
    m.u = Param(m.I, initialize=param_dicts['u']) # base deliciousness
    m.r = Param(m.I, initialize=param_dicts['r']) # rarity score
    m.w = Param(m.I, initialize=param_dicts['w']) # stomach filling coefficient
    m.p = Param(m.I, initialize=param_dicts['p']) # discomfort coefficient
    m.f = Param(m.I, initialize=param_dicts['f']) # price per portion
    m.alpha = Param(initialize=alpha) # rarity weight
    m.Wmax  = Param(initialize=1.25) # stomach capacity
    m.Pmax  = Param(initialize=1) # discomfort tolerance
    m.F     = Param(initialize=15) # fixed entry price

    # Constraints
    m.constraints = ConstraintList()

    # Stomach Capacity Constraint
    m.constraints.add(sum(m.w[i]*m.x[i] for i in m.I) <= m.Wmax)

    # Discomfort Tolerance Constraint 
    m.constraints.add(sum(m.p[i]*m.x[i] for i in m.I) <= m.Pmax)

    # Value Recovery Constraint
    m.constraints.add(sum(m.f[i]*m.x[i] for i in m.I) >= m.F)

    # Objective function
    if function_type == 'logarithmic':
        def obj_rule(m):
            return sum((m.u[i] + m.alpha * m.r[i]) * log(1 + m.x[i]) for i in m.I)
    elif function_type == 'linear':
        def obj_rule(m):
            return sum((m.u[i] + m.alpha * m.r[i]) * m.x[i] for i in m.I)

    m.obj = Objective(rule=obj_rule, sense=maximize)

    # Solving
    print(f"--- Solving for {function_type} utility, alpha = {alpha}  ---")

    solver  = SolverFactory('ipopt',executable=r'C:\Users\arnau\miniconda3\Library\bin\ipopt.exe')
    results = solver.solve(m, tee=False)
    print("Termination condition:", results.solver.termination_condition)

    # Show results
    model_vars = list(m.component_data_objects(Var)) # list of variables

    # Save results in data frame
    df_column_portions = f'optimal portions alpha={alpha}, {function_type}'
    df_results[df_column_portions] = [model_vars[i]() for i in range(len(model_vars))]

    # Format for readability
    def int_zeros(x):
        if x==0.0:
            return str(0).format('.2g')
        else:
            return x

    df_results[df_column_portions] = df_results[df_column_portions].round(1).apply(int_zeros)


    print(f"Objective value = {m.obj():.3g}")
    print('\n')

# Save results to CSV
df_results.to_csv('results.csv')

0.5 logarithmic
--- Solving for logarithmic utility, alpha = 0.5  ---
Termination condition: optimal
Objective value = 57


1 logarithmic
--- Solving for logarithmic utility, alpha = 1  ---
Termination condition: optimal
Objective value = 68.8


1 linear
--- Solving for linear utility, alpha = 1  ---
Termination condition: optimal
Objective value = 194


2.0 logarithmic
--- Solving for logarithmic utility, alpha = 2.0  ---
Termination condition: optimal
Objective value = 92.9


